In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install wandb

Looking in indexes: https://download.pytorch.org/whl/cu118


# **Question 1 & Question 2**

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split, ConcatDataset
from torchvision.datasets import ImageFolder
import wandb
from wandb.sdk.wandb_run import Run
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np


classes = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]
# Check if CUDA (GPU) is available, and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Set up data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load the dataset
train_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/train', transform=train_transforms)
test_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/val', transform=test_transforms)


In [3]:
import torch
print(torch.cuda.is_available())
print(device)

True
cuda


In [4]:
# Count the number of samples in each class
class_counts = {}
pbar = tqdm(total=len(train_data))
for _, label in train_data:
    if label not in class_counts:
        class_counts[label] = 0
    class_counts[label] += 1
    pbar.set_postfix()
    pbar.update(1)

pbar.close()

# Calculate the number of samples per class for validation set
val_size_per_class = {label: int(count * 0.2) for label, count in class_counts.items()}

# Initialize lists to hold indices for train and validation sets
train_indices = []
val_indices = []

# Iterate through the dataset and assign samples to train or validation set
pbar = tqdm(total=len(train_data))
for idx, (_, label) in enumerate(train_data):
    if val_size_per_class[label] > 0:
        val_indices.append(idx)
        val_size_per_class[label] -= 1
    else:
        train_indices.append(idx)
    pbar.set_postfix()
    pbar.update(1)

pbar.close()

# Create SubsetRandomSampler for train and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
    

  0%|          | 0/9999 [00:00<?, ?it/s]

  0%|          | 0/9999 [00:00<?, ?it/s]

In [ ]:
# print(len(train_loader.dataset))
# print(len(val_loader.dataset))
# print(len(val_indices))

In [ ]:
# data_augmentation = True
# batch_size = 32
# if data_augmentation:
#     additional_transforms = transforms.Compose([
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(10),
#         transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
#         transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
#     ])

#     # Apply additional transformations to the new DataLoader
#     original_dataset, transformed_dataset = apply_additional_transforms(train_loader, additional_transforms, batch_size)
#     combined_dataset = ConcatDataset([original_dataset, transformed_dataset])

#     # Create a new DataLoader using the combined dataset
#     combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)

# print(len(combined_loader))

In [5]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, filters, activation, filter_organization, data_augmentation, batch_norm, dropout):
        super(CNN, self).__init__()

        self.conv_layers = nn.Sequential()
        self.dense_layers = nn.Sequential()

        # Add conv layers based on filter organization
        if filter_organization == 'same':
            num_filters = [filters] * 5  # Assuming 5 convolution layers
        elif filter_organization == 'double':
            num_filters = [filters * 2**i for i in range(5)]  # Doubling filters in each subsequent layer
        else:
            num_filters = [filters // 2**i for i in range(5)]  # Halving filters in each subsequent layer

        in_channels = 3
        for i, f in enumerate(num_filters):
            self.conv_layers.add_module(f"conv_{i}", nn.Conv2d(in_channels, f, kernel_size=3, padding=1))
            if batch_norm:
                self.conv_layers.add_module(f"batch_norm_{i}", nn.BatchNorm2d(f))
            self.conv_layers.add_module(f"{activation}_{i}", getattr(nn, activation)())
            self.conv_layers.add_module(f"maxpool_{i}", nn.MaxPool2d(kernel_size=2))
            self.conv_layers.add_module("dropout", nn.Dropout2d(p=dropout)) 
            in_channels = f

        self.dense_layers.add_module("flatten", nn.Flatten())
        self.dense_layers.add_module("dense", nn.Linear(in_channels * 7 * 7, 512))
        self.dense_layers.add_module("relu", nn.ReLU())
        self.output_layer = nn.Linear(512, len(train_data.classes))

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.dense_layers(x)
        x = self.output_layer(x)
        return x

In [6]:
# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

In [7]:
# Training loop
def training_model(epochs, optimizer, criterion, model, train_loader, val_loader):
    for epoch in range(epochs):
        model.train()
        training_loss = 0.0
        train_accuracy = 0.0
        pbar = tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}')
        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device) 
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            train_accuracy += calculate_accuracy(outputs, labels)
            pbar.set_postfix({'Train Loss': training_loss / (pbar.n + 1), 'Train Acc': train_accuracy / (pbar.n + 1)})
            pbar.update(1)

        pbar.close()


        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device) 
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                val_accuracy += calculate_accuracy(outputs, labels)

        train_accuracy /= len(train_loader)
        training_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        print(f'Epoch {epoch+1}/{epochs}, Train_Loss: {training_loss:.4f},  Train_Acc: {train_accuracy:.4f},  Val_Loss: {val_loss:.4f},  Val_Accuracy: {val_accuracy:.4f}')
        wandb.log({"epoch": epoch+1, "train_loss": training_loss, "val_loss": val_loss, "val_accuracy": val_accuracy, "train_accuracy": train_accuracy})
    return model



# **Question 3**

In [8]:
sweep_config = {
    'method': 'bayes',  # Random search method
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},  # Metric to optimize
    'parameters': {
        'epochs': {'values':[5, 10]},
        'batch_size': {'values':[32, 64]},
        'num_filters': {'values': [8, 16, 32, 64]},
        'activation': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish']},
        'filter_organization': {'values': ['same', 'double', 'halve']},
        'data_augmentation': {'values': [True, False]},
        'batch_norm': {'values': [True, False]},
        'dropout': {'values': [0.2, 0.3]}
    }
    
}

#Best Configuration
# sweep_config = {
#     'method': 'bayes',  # Random search method
#     'metric': {'goal': 'maximize', 'name': 'val_accuracy'},  # Metric to optimize
#     'parameters': {
#         'epochs': {'values':[10]},
#         'batch_size': {'values':[32]},
#         'num_filters': {'values': [128]},
#         'activation': {'values': ['GELU']},
#         'filter_organization': {'values': ['same']},
#         'data_augmentation': {'values': [False]},
#         'batch_norm': {'values': [True]},
#         'dropout': {'values': [0.3]}
#     }
    
# }

In [9]:
def apply_additional_transforms(loader, additional_transforms, batch_size):
    transformed_dataset = []
    original_dataset = []
    pbar = tqdm(total=len(loader))
    for images, labels in loader:
        images1 = additional_transforms(images)
        for i in range(batch_size):
            original_dataset.append((images[i], labels[i]))
            transformed_dataset.append((images1[i], labels[i]))
        pbar.set_postfix()
        pbar.update(1)

    pbar.close()
    return original_dataset, transformed_dataset



In [10]:
def augment_data(data_augmentation, train_loader, batch_size):
    if data_augmentation:
        additional_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        ])

        # Apply additional transformations to the new DataLoader
        original_dataset, transformed_dataset = apply_additional_transforms(train_loader, additional_transforms, batch_size)
        combined_dataset = ConcatDataset([original_dataset, transformed_dataset])

        # Create a new DataLoader using the combined dataset
        combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
    else:
        combined_loader = train_loader
    return combined_loader

In [11]:
def train_CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout, batch_size, epochs):
    # Create an instance of the CNN model
    model = CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout)
    model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Create DataLoader instances for train and validation sets using the samplers
    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_data, batch_size=batch_size, sampler=val_sampler)

    combined_loader = augment_data(data_augmentation, train_loader, batch_size)
    
    model = training_model(epochs, optimizer, criterion, model, combined_loader, val_loader)
    return model

In [12]:
# wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
# sweep_id = wandb.sweep(sweep_config, project='DL-Assignment-2')

# # Define your training function
# def train():
    
#     # Initialize Wandb run with custom run name
#     with wandb.init() as run:
        
#         # Use wandb.config to access hyperparameters in your training script
#         config = wandb.config
#         num_filters = config['num_filters']
#         activation = config['activation']
#         filter_organization = config['filter_organization']
#         data_augmentation = config['data_augmentation']
#         batch_norm = config['batch_norm']
#         batch_size = config['batch_size']
#         epochs = config['epochs']
#         dropout = config['dropout']
#         # Generate a custom run name based on hyperparameters
#         run_name = "epochs_" + str(epochs) + "_nFilters_" + str(num_filters) + "_activation_" + str(activation)+ "_filterOrg_" + str(filter_organization) + "_batchSize_" + str(batch_size)
#         wandb.run.name = run_name
        
#         model = train_CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout, batch_size, epochs)

    
        
# # Run the sweep
# wandb.agent(sweep_id, function=train, count=20)
# wandb.finish()

# **Question 4**

In [13]:
wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
with wandb.init( project='DL-Assignment-2') as run: 
    
    #Best Configuration
    epochs = 10
    batch_size = 32
    num_filters = 128
    activation = 'GELU'
    filter_organization = 'same'
    data_augmentation = False
    batch_norm = True 
    dropout = 0.3

    run_name = "epochs_" + str(epochs) + "_nFilters_" + str(num_filters) + "_activation_" + str(activation)+ "_filterOrg_" + str(filter_organization) + "_batchSize_" + str(batch_size)
    wandb.run.name = run_name
    
    model = train_CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout, batch_size, epochs)

wandb.finish()

# Test the model
test_loader = DataLoader(test_data, batch_size=batch_size)
model.eval()

test_accuracy = 0.0
results_list = []
pbar = tqdm(total=len(test_loader))
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) 
        outputs = model(images)
        images.to("cpu")
        labels.to("cpu")
        for i in range(len(images)):
            image = images[i]
            label = labels[i]
            output = outputs[i].argmax(dim = 0)
            if (label == output):
                test_accuracy += 1
            result_tuple = (image, label, output)
            results_list.append(result_tuple)
        pbar.set_postfix()
        pbar.update(1)

pbar.close()
wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
with wandb.init( project='DL-Assignment-2') as run:      
    run_name = "test_accuracy"
    wandb.run.name = run_name
    test_accuracy /= len(test_data)
    print(test_accuracy)
    wandb.log({"test_accuracy": test_accuracy})

wandb.finish()


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23m047. Use `wandb login --relogin` to force relogin


Epoch 1/10:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1/10, Train_Loss: 2.1886,  Train_Acc: 0.2221,  Val_Loss: 2.1086,  Val_Accuracy: 0.2729


Epoch 2/10:   0%|          | 0/250 [00:00<?, ?it/s]

Epoch 2/10, Train_Loss: 1.9936,  Train_Acc: 0.2920,  Val_Loss: 2.0160,  Val_Accuracy: 0.2920


...

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▄▅▅▆▇▇█
train_loss,█▆▅▅▄▄▃▂▂▁
val_accuracy,▁▃▃▄▅▇▆██▇
val_loss,█▅▄▃▂▁▂▁▁▄
epoch,10
train_accuracy,0.47037
train_loss,1.49777
val_accuracy,0.34167
val_loss,1.95378


  0%|          | 0/63 [00:00<?, ?it/s]

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


0.361


test_accuracy,▁
test_accuracy,0.361


In [41]:
results_dict = {}

# Organize tuples by label value in a dictionary
pbar = tqdm(total=len(results_list))
for image, label, predicted in results_list:
    image = image.to("cpu")
    label = label.to("cpu")
    predicted = predicted.to("cpu")
    if label not in results_dict:
        results_dict[label] = []
    results_dict[label].append((image, label, predicted))
    pbar.set_postfix()
    pbar.update(1)

pbar.close()
# a = 0
# a = torch.tensor(a).to(device)

# b = 1
# b = torch.tensor(b).to(device)
# print(len(results_dict[a]), len(results_dict[b]))

# Create a list of three tuples for each label value

pbar = tqdm(total=len(results_dict))
l = [0, 0 ,0, 0,0,0,0,0,0,0]
labels_done = []
filtered_results_list = []
count = 0
for label, tuples_list in results_dict.items():
    if label.item() in labels_done and l[label.item()] < 3:
#         print(tuples_list[0])
        filtered_results_list.append(tuples_list)
        l[label.item()] += 1
    elif label.item() not in labels_done:
        labels_done.append(label.item())
        filtered_results_list.append(tuples_list)
        l[label.item()] = 1

pbar.close()

print(len(filtered_results_list))


wandb.login(key = "1d2c93cf7ddd48a63114848b66796301171827b6")
wandb.init(project='DL-Assignment-2')

# Create a figure and axis objects for plotting
fig, axs = plt.subplots(10, 3, figsize=(12, 40))

# Flatten the axis array to simplify indexing
axs = axs.flatten()

for idx, i in enumerate(filtered_results_list):
    # Assuming i[0] contains the image data in the correct format (e.g., a NumPy array)
    image_data = i[0][0].numpy()  # Convert tensor to NumPy array

    # Transpose image data if needed (e.g., for channels-first format)
    image_transposed = np.transpose(image_data, (1, 2, 0))

    # Display the image on the corresponding subplot
    axs[idx].imshow(image_transposed)
    axs[idx].axis('off')
    axs[idx].set_title(f"Label: {classes[i[0][1]]}\nPredicted: {classes[i[0][2]]}")

# Adjust layout to prevent overlap of titles
plt.tight_layout()

# Log the plot to wandb as an image
wandb.log({"image_grid": wandb.Image(plt)})
plt.close()


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


30


In [ ]:
# !jupyter notebook --ServerApp.iopub_msg_rate_limit=2000.0